# Libraries

In [5]:
import pandas as pd
import numpy as np

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [99]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout
import keras_tuner as kt

# Data

In [8]:
df= pd.read_csv('diabetes.csv')
df.head(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
df.corr(numeric_only=True)['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [10]:
x= df.iloc[:,0:-1].values
y=df.iloc[:,-1].values

# Modeing and Handling Data

In [11]:
scaler= StandardScaler()
x=scaler.fit_transform(x)

In [12]:
x.shape

(768, 8)

In [13]:
train_df, test_df, train_target, test_target= train_test_split(x,y, test_size=0.2, random_state=1)

In [14]:
model= Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
model.fit(train_df, train_target, batch_size=32, epochs=100, validation_data=(test_df,test_target))

Epoch 1/100
20/20 [==============================] - 1s 19ms/step - loss: 0.6865 - accuracy: 0.5700 - val_loss: 0.6407 - val_accuracy: 0.5974
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6408 - accuracy: 0.6287 - val_loss: 0.6005 - val_accuracy: 0.6948
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6068 - accuracy: 0.6840 - val_loss: 0.5697 - val_accuracy: 0.7403
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5771 - accuracy: 0.7068 - val_loss: 0.5458 - val_accuracy: 0.7468
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.5550 - accuracy: 0.7117 - val_loss: 0.5262 - val_accuracy: 0.7597
Epoch 6/100
20/20 [==============================] - 0s 10ms/step - loss: 0.5360 - accuracy: 0.7166 - val_loss: 0.5097 - val_accuracy: 0.7597
Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5215 - accuracy: 0.7280 - val_loss: 0.5004 - val_accuracy: 0.7727
Epoch 8/100

# Optimizer tuner

In [21]:
def build_model(hp):

    model= Sequential()
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    optimizer=hp.Choice('optimizer',values= ['adam', 'sgd', 'rmsprop', 'adadelta'])

    model.compile(optimizer= optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [22]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

In [23]:
tuner.search(train_df, train_target, epochs=5, validation_data=(test_df, test_target))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 12s


In [26]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [27]:
model= tuner.get_best_models(num_models=1)[0]

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit(train_df, train_target, batch_size=32, epochs=100, initial_epoch=6, validation_data=(test_df, test_target))

Epoch 7/100
20/20 [==============================] - 2s 22ms/step - loss: 0.5084 - accuracy: 0.7720 - val_loss: 0.4875 - val_accuracy: 0.8052
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4949 - accuracy: 0.7720 - val_loss: 0.4769 - val_accuracy: 0.8247
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4846 - accuracy: 0.7736 - val_loss: 0.4687 - val_accuracy: 0.8247
Epoch 10/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4763 - accuracy: 0.7785 - val_loss: 0.4623 - val_accuracy: 0.8182
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4695 - accuracy: 0.7785 - val_loss: 0.4572 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4644 - accuracy: 0.7785 - val_loss: 0.4546 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4597 - accuracy: 0.7818 - val_loss: 0.4522 - val_accuracy: 0.8182
Epoch 14

# Neurons tuner

In [49]:
def build_model(hp):

    model=Sequential()

    unit= hp.Int('uints', 8, 256)

    model.add(Dense(units=unit, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss= 'binary_crossentropy', metrics=['accuracy'])

    return model

In [50]:
tuner=kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='mydir', project_name='new test')

In [51]:
tuner.search(train_df, train_target, epochs= 5, validation_data=(test_df, test_target))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 10s


In [52]:
tuner.get_best_hyperparameters()[0].values

{'uints': 230}

In [53]:
model= tuner.get_best_models(num_models=1)[0]

In [54]:
model.fit(train_df, train_target, batch_size=32, epochs=100, initial_epoch=6 )

Epoch 7/100
20/20 [==============================] - 1s 2ms/step - loss: 0.4919 - accuracy: 0.7638
Epoch 8/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4741 - accuracy: 0.7785
Epoch 9/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4658 - accuracy: 0.7736
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4600 - accuracy: 0.7818
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4547 - accuracy: 0.7785
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4537 - accuracy: 0.7801
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4498 - accuracy: 0.7704
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4479 - accuracy: 0.7752
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4453 - accuracy: 0.7801
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4445 - accuracy: 0.7850
Epo

# Layer tuner

In [83]:
def build_model(hp):

    model=Sequential()
    model.add(Dense(230, activation='relu', input_dim=8))

    for i in range(hp.Int('layers', min_value=1, max_value=10)):
        model.add(Dense(230, activation='relu'))
 
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop', loss= 'binary_crossentropy', metrics=['accuracy'])

    return model

In [84]:
tuner= kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, directory='mydir', project_name='layers_4')

In [85]:
tuner.search(train_df, train_target, epochs=5, validation_data= (test_df, test_target))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 19s


In [86]:
tuner.get_best_hyperparameters()[0].values

{'layers': 6}

In [87]:
model= tuner.get_best_models(num_models=1)[0]

In [88]:
model.fit(train_df, train_target, epochs=100, initial_epoch=6, validation_data=(test_df,test_target))

Epoch 7/100
20/20 [==============================] - 2s 22ms/step - loss: 0.4735 - accuracy: 0.7736 - val_loss: 0.4741 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4487 - accuracy: 0.7834 - val_loss: 0.5821 - val_accuracy: 0.7208
Epoch 9/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4336 - accuracy: 0.7883 - val_loss: 0.4871 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 12ms/step - loss: 0.4453 - accuracy: 0.7899 - val_loss: 0.4897 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4115 - accuracy: 0.8127 - val_loss: 0.5934 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 11ms/step - loss: 0.4229 - accuracy: 0.8078 - val_loss: 0.5137 - val_accuracy: 0.7792
Epoch 13/100
20/20 [==============================] - 0s 11ms/step - loss: 0.3891 - accuracy: 0.8176 - val_loss: 0.5286 - val_accuracy: 0.7792
Epoc

# Tuning all

In [122]:
def build_model(hp):

    model=Sequential()
    counter=0

    for i in range(hp.Int('layers', min_value=1, max_value=10)):
        model.add(Dense(230, activation='relu'))
        if counter==0:
            model.add(Dense(
                            hp.Int('units'+str(i) , min_value=8, max_value=128, step=8), 
                            activation= hp.Choice('activation'+str(i), values=['relu', 'tanh', 'sigmoid']),
                            input_dim=8
                            )
                    )
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        else:
            model.add(Dense(
                            hp.Int('units'+str(i) , min_value=8, max_value=128, step=8), 
                            activation= hp.Choice('activation'+str(i), values=['relu', 'elu', 'tanh','sigmoid'])
                            )
                    )
            model.add(Dropout(hp.Choice('dropout'+str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        
        counter+=1

    model.add(Dense(1, activation='sigmoid'))
    model.compile(
                    optimizer=hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam', 'adamdelta', 'adagrad']),
                    loss= 'binary_crossentropy', 
                    metrics=['accuracy']
                )

    return model

In [123]:
tuner= kt.RandomSearch(build_model, 
                       objective='val_accuracy',
                       max_trials=5,
                       directory='mydir',
                       project_name='final_7'
                       )

In [124]:
tuner.search(train_df, train_target, epochs=5, validation_data=(test_df, test_target))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 16s


In [125]:
tuner.get_best_hyperparameters()[0].values

{'layers': 2,
 'units0': 72,
 'activation0': 'relu',
 'dropout0': 0.1,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'sigmoid',
 'dropout1': 0.7,
 'units2': 24,
 'activation2': 'elu',
 'dropout2': 0.3,
 'units3': 32,
 'activation3': 'tanh',
 'dropout3': 0.6,
 'units4': 72,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 96,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 16,
 'activation6': 'sigmoid',
 'dropout6': 0.1}

In [127]:
model= tuner.get_best_models(num_models=1)[0]

In [128]:
model.fit(train_df, train_target, epochs=200, initial_epoch=5, validation_data=(test_df, test_target))

Epoch 6/200
20/20 [==============================] - 1s 19ms/step - loss: 0.5666 - accuracy: 0.7394 - val_loss: 0.5020 - val_accuracy: 0.8052
Epoch 7/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5514 - accuracy: 0.7296 - val_loss: 0.4943 - val_accuracy: 0.8052
Epoch 8/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5456 - accuracy: 0.7655 - val_loss: 0.4911 - val_accuracy: 0.7987
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5553 - accuracy: 0.7410 - val_loss: 0.4899 - val_accuracy: 0.7922
Epoch 10/200
20/20 [==============================] - 0s 5ms/step - loss: 0.5390 - accuracy: 0.7378 - val_loss: 0.4920 - val_accuracy: 0.7727
Epoch 11/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5588 - accuracy: 0.7248 - val_loss: 0.4909 - val_accuracy: 0.7987
Epoch 12/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5477 - accuracy: 0.7362 - val_loss: 0.4866 - val_accuracy: 0.7922
Epoch 13/

In [130]:
tuner.get_best_hyperparameters()[0].values

{'layers': 2,
 'units0': 72,
 'activation0': 'relu',
 'dropout0': 0.1,
 'optimizer': 'adam',
 'units1': 8,
 'activation1': 'sigmoid',
 'dropout1': 0.7,
 'units2': 24,
 'activation2': 'elu',
 'dropout2': 0.3,
 'units3': 32,
 'activation3': 'tanh',
 'dropout3': 0.6,
 'units4': 72,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 96,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 16,
 'activation6': 'sigmoid',
 'dropout6': 0.1}

# Model 1

In [131]:
model=Sequential()

model.add(Dense(72, activation='relu', input_dim=8))
model.add(Dropout(0.1))

model.add(Dense(8, activation='sigmoid'))
model.add(Dropout(0.7))

model.add(Dense(24, activation='elu'))
model.add(Dropout(0.3))

model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.6))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [132]:
model.fit(train_df, train_target, epochs=200, validation_data=(test_df, test_target))

Epoch 1/200
20/20 [==============================] - 1s 17ms/step - loss: 0.7321 - accuracy: 0.5798 - val_loss: 0.6558 - val_accuracy: 0.6429
Epoch 2/200
20/20 [==============================] - 0s 6ms/step - loss: 0.7038 - accuracy: 0.6026 - val_loss: 0.6505 - val_accuracy: 0.6429
Epoch 3/200
20/20 [==============================] - 0s 6ms/step - loss: 0.7019 - accuracy: 0.5814 - val_loss: 0.6437 - val_accuracy: 0.6429
Epoch 4/200
20/20 [==============================] - 0s 6ms/step - loss: 0.7044 - accuracy: 0.5912 - val_loss: 0.6408 - val_accuracy: 0.6429
Epoch 5/200
20/20 [==============================] - 0s 6ms/step - loss: 0.7026 - accuracy: 0.5798 - val_loss: 0.6386 - val_accuracy: 0.6429
Epoch 6/200
20/20 [==============================] - 0s 6ms/step - loss: 0.7027 - accuracy: 0.5977 - val_loss: 0.6363 - val_accuracy: 0.6429
Epoch 7/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6915 - accuracy: 0.5961 - val_loss: 0.6332 - val_accuracy: 0.6429
Epoch 8/200


# Model 2

In [133]:
model=Sequential()

model.add(Dense(72, activation='relu', input_dim=8))
model.add(Dropout(0.1))

model.add(Dense(24, activation='elu'))
model.add(Dropout(0.3))

model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.6))

model.add(Dense(72, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [134]:
model.fit(train_df, train_target, epochs=200, validation_data=(test_df, test_target))

Epoch 1/200
20/20 [==============================] - 1s 18ms/step - loss: 0.6372 - accuracy: 0.6384 - val_loss: 0.5867 - val_accuracy: 0.7143
Epoch 2/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5982 - accuracy: 0.7020 - val_loss: 0.5284 - val_accuracy: 0.7922
Epoch 3/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5257 - accuracy: 0.7394 - val_loss: 0.4949 - val_accuracy: 0.8052
Epoch 4/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5299 - accuracy: 0.7329 - val_loss: 0.4857 - val_accuracy: 0.8052
Epoch 5/200
20/20 [==============================] - 0s 9ms/step - loss: 0.5000 - accuracy: 0.7443 - val_loss: 0.4755 - val_accuracy: 0.7987
Epoch 6/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5048 - accuracy: 0.7476 - val_loss: 0.4711 - val_accuracy: 0.7987
Epoch 7/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5026 - accuracy: 0.7476 - val_loss: 0.4684 - val_accuracy: 0.8182
Epoch 8/200


# Model 3

In [135]:
model=Sequential()

model.add(Dense(72, activation='relu', input_dim=8))
model.add(Dropout(0.1))

model.add(Dense(32, activation='tanh'))
model.add(Dropout(0.6))

model.add(Dense(72, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(96, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [136]:
model.fit(train_df, train_target, epochs=200, validation_data=(test_df, test_target))

Epoch 1/200
20/20 [==============================] - 2s 21ms/step - loss: 0.6619 - accuracy: 0.6401 - val_loss: 0.6414 - val_accuracy: 0.6429
Epoch 2/200
20/20 [==============================] - 0s 6ms/step - loss: 0.6155 - accuracy: 0.6629 - val_loss: 0.5779 - val_accuracy: 0.7013
Epoch 3/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5681 - accuracy: 0.6987 - val_loss: 0.5227 - val_accuracy: 0.8117
Epoch 4/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5254 - accuracy: 0.7215 - val_loss: 0.4959 - val_accuracy: 0.7987
Epoch 5/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4951 - accuracy: 0.7590 - val_loss: 0.4843 - val_accuracy: 0.8117
Epoch 6/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5060 - accuracy: 0.7655 - val_loss: 0.4827 - val_accuracy: 0.8247
Epoch 7/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4758 - accuracy: 0.7720 - val_loss: 0.4813 - val_accuracy: 0.8247
Epoch 8/200


# Model 4

In [137]:
model=Sequential()

model.add(Dense(72, activation='relu', input_dim=8))
model.add(Dropout(0.1))

model.add(Dense(72, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(96, activation='relu'))
model.add(Dropout(0.1))


model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [138]:
model.fit(train_df, train_target, epochs=200, validation_data=(test_df, test_target))

Epoch 1/200
20/20 [==============================] - 1s 18ms/step - loss: 0.6726 - accuracy: 0.5896 - val_loss: 0.6222 - val_accuracy: 0.6429
Epoch 2/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5873 - accuracy: 0.6710 - val_loss: 0.5441 - val_accuracy: 0.6948
Epoch 3/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5192 - accuracy: 0.7362 - val_loss: 0.4786 - val_accuracy: 0.8117
Epoch 4/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4800 - accuracy: 0.7785 - val_loss: 0.4582 - val_accuracy: 0.8182
Epoch 5/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4708 - accuracy: 0.7801 - val_loss: 0.4534 - val_accuracy: 0.8117
Epoch 6/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4646 - accuracy: 0.7850 - val_loss: 0.4531 - val_accuracy: 0.8052
Epoch 7/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4632 - accuracy: 0.7671 - val_loss: 0.4520 - val_accuracy: 0.7987
Epoch 8/200


# Model 5

In [139]:
model=Sequential()

model.add(Dense(72, activation='relu', input_dim=8))
model.add(Dropout(0.1))

model.add(Dense(72, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(24, activation='elu'))
model.add(Dropout(0.3))


model.add(Dense(96, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [140]:
model.fit(train_df, train_target, epochs=200, validation_data=(test_df, test_target))

Epoch 1/200
20/20 [==============================] - 2s 26ms/step - loss: 0.6523 - accuracy: 0.6221 - val_loss: 0.5995 - val_accuracy: 0.6494
Epoch 2/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5789 - accuracy: 0.6726 - val_loss: 0.5327 - val_accuracy: 0.8052
Epoch 3/200
20/20 [==============================] - 0s 6ms/step - loss: 0.5116 - accuracy: 0.7345 - val_loss: 0.4927 - val_accuracy: 0.8117
Epoch 4/200
20/20 [==============================] - 0s 7ms/step - loss: 0.4840 - accuracy: 0.7687 - val_loss: 0.4779 - val_accuracy: 0.7987
Epoch 5/200
20/20 [==============================] - 0s 6ms/step - loss: 0.4765 - accuracy: 0.7687 - val_loss: 0.4778 - val_accuracy: 0.8052
Epoch 6/200
20/20 [==============================] - 0s 7ms/step - loss: 0.4513 - accuracy: 0.7899 - val_loss: 0.4824 - val_accuracy: 0.7987
Epoch 7/200
20/20 [==============================] - 0s 7ms/step - loss: 0.4553 - accuracy: 0.7752 - val_loss: 0.4789 - val_accuracy: 0.8052
Epoch 8/200
